In [1]:
import numpy as np
import pandas as pd

In [ ]:
class RNN:
    def __init__(self,hidden_size,vocab_size,seq_length,learning_rate):
        #hyper parameters
        self.hidden_size=hidden_size
        self.vocab_size=vocab_size
        self.seq_length=seq_length
        self.learning_rate=learning_rate
        
        #model parameters
        #np.random.uniform(low,high,size): low부터 high까지의 난수를 size 크기 만큼 생성
        #1/vocab_size가 아닌 1./vocab_size인 이유: 1이 아닌 '1.'으로 명시적으로 실수형 연산 지정함
        self.U=np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hiddem_sie, vocab_size))
        self.V=np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W=np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        
        #bias for hidden layer
        self.b = np.zeros((hidden_size,1)) # bias for hidden layer.
        #bias for output
        self.c = np.zeros((vocab_size,1)) # bias for output.

In [ ]:
def forward(self, inputs, hprev):
    xs, hs, os, yacp={}, {}, {}, {}
    hs[-1]=np.copy(hprev)

    for t in range(len(inputs)):
        #xs[t]: 현재 시점 t의 입력 벡터(크기:(vocab_size,1))
        xs[t]=zero_init(self.vocab_size, 1)
        #ont-hot 인코딩
        xs[t][inputs[t]]=1

        #<hidden state>
        #hs[t]: 현재 시점 t의 은닉 상태, np.tanh(): 하이퍼볼릭 탄젠트 함수(비선형 활성화 함수)
        #hs[t-1]: 이전 시점 t-1의 은닉 상태(크기: (hidden_size,1))
        #self.U: 입력 가중치 행렬(크기: (hidden_size, vocab_size))
        #self.W: 은닉 상태 가중치 행렬(크기: (hidden_size, hidden_size))

        #np.dot(self.U, xs[t]): 입력 xs[t]에 대한 선형 변환(입력 가중치 적용) / 현재 입력 xs[t]가 은닉 상태에 어떤 영향을 미치는지 계산
        #np.dot(self.W, hs[t-1]): 이전 은닉 상태 hs[t-1]에 대한 선형 변환(은닉 상태 가중치 적용) / 이전 은닉 상태 hs[t-1]이 현재 은닉 상태에 어떤 영향을 미치는지 계산

        hs[t]=np.tanh(np.dot(self.U, xs[t])+np.dot(self.W, hs[t-1])+self.b)

        #<output> : 은닉 상태 hs[t]가 출력 os[t]로 바뀜
        #os[t]: RNN의 출력층 값
        #self.V: 출력 가중치 행렬(크기: (output_size, hidden_size))
        os[t]=np.dot(self.V, hs[t])+self.c

        #ycap[t]: 최종 확률 분포(예측값), 소프트맥스 함수로 0~1 사이 값으로 변환
        ycap[t]=self.softmax(os[t])

    return xs, hs, os, ycap 